Develop learning_wrapper algorithm

Utilize the combinations of UserBatch (class) and learning_dist_metrics

[pseudocode active weighting vector learning algorithm]
*step 1*: compute the aggregate distance information with argument of weighting vector [w]:
        a. sum of weighted distances of friends
        b. sum of weighted distances of non-friends
                               
*step 2*: argmin( [w] ) = sum(distances of same class) / sum(distances of different classes)
 
*step 3*: test hypothesis that location difference between friend-distance distribution
        vs. non-friend distance per user
                               
*step 4*: group users based on the previous test (paramer: distribution type, compared non-friend sample size):
        a. a user is assigned to group 1, if its two-type distance distributions location
           are significantly different;
        b. a user is assigned to group 2, if its two-type distance distributions location
           are not significantly different.
               
*step 5*: apply the 1-4 steps on users of group 1, split users of this group into: group 1 (with siganificant
        difference) and group 3 (with insignificant difference);
        Similiarly, apply 1-4 steps on users of group 2, split users of this group into: group 2 (with significant
        difference) and group 3 (with insignificant diference).
 
*step 6*: repeat 5th step until the stoping threshold is met:
        a. fixed iteration, or;
        b. insubstantial change in either weighting vector, or;
        c. insubstantial change in members in group 1-3.

In [6]:
import os
import sys
import glob

import numpy as np
import scipy as sp
import pandas as pd

from learning_dist_metrics.ldm import LDM
from learning_dist_metrics.datasets import load_data
from UserBatch import SimScore, WeightedEuclidean, UserBatch

import matplotlib.pyplot as plt
%matplotlib inline

load the simulate data sets, the frienships were generated based on "hand-shaking" protocol.

In [15]:
DATA_PATH = "./data/sim_data_yi/" 

users_df   = pd.read_csv(DATA_PATH + "users_profile.csv", header = 0, sep = ",")
friends_df = pd.read_csv(DATA_PATH + "friendships.csv", header = 0, sep = ",")
dist_df    = pd.read_csv(DATA_PATH + "dist_mat.csv", header = 0, sep = ",")

In [16]:
## friends_df is processed
## a. create a new column to denote the user pair
## b. exclude user-pair of non-friends
## c. drop the 'isFriend' columns
friends_df = friends_df[friends_df.isFriend == 1]
friends_df["pair"] = friends_df[["uid_a", "uid_b"]].apply(lambda x: (int(x[0]), int(x[1])), axis=1)
friends_df.drop("isFriend", axis=1, inplace=True)
friends_df = friends_df[["pair", "uid_a", "uid_b"]]
friends_df.head(3)

,pair,uid_a,uid_b
1,"(0, 2)",0,2
3,"(0, 4)",0,4
4,"(0, 5)",0,5


In [ ]:
sim_ubatch = UserBatch()
sim_ubatch.load_s